In [2]:
%pip install transformers
%pip install torch
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import necessary libraries
from transformers import BertTokenizer, BertForTokenClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader 
from tqdm import tqdm 
import torch

In [4]:
# Assuming a predefined set of entity types
entity_types = ["O", "B_REF_IN", "I_REF_IN"]
# Set num_labels
num_labels = len(entity_types)
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') 
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
# Define batch_size
batch_size = 32  # Adjust as needed
# Define learning rate
learning_rate = 5e-5  # Adjust as needed

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Sample data from CoNLL-2003 (Replace this with your dataset) 

# train_dataset_sample = 
# [
#     {"text": "John works at Google in New York.", "labels":
# {"entities": [(0, 4, "PER"), (14, 20, "ORG"), (24, 32, "LOC")]}},
#     {"text": "Apple Inc. is a technology company.", "labels":
# {"entities": [(0, 10, "ORG")]}},     # Add more samples as needed
# ]

In [6]:
def tokenize_and_format_data(dataset, tokenizer):
    tokenized_data = []
    for sample in dataset:
        text = sample["text"]
        entities = sample["label"]
        # Tokenize the input text using the BERT tokenizer

        tokens =  tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))
        print("tokens",tokens)
        # Initialize labels for each token as 'O' (Outside)
        labels = ['O'] * len(tokens)
        # Update labels for entity spans
        for start, end, entity_type in entities:
            # Tokenize the prefix to get the correct offset
            prefix_tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text[:start])))
            print("prefix tokens", prefix_tokens)
            start_token = len(prefix_tokens) - 1
            # Tokenize the entity to get its length
            entity_tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text[start:end])))
            print("entity tokens",entity_tokens)
            end_token = start_token + len(entity_tokens) - 2
            labels[start_token] = f"B_{entity_type}"
            print(start_token, end_token, labels)
            for i in range(start_token + 1, end_token):
                labels[i] = f"I_{entity_type}"
            
                # Convert tokens and labels to input IDs and label IDs
                input_ids = tokenizer.convert_tokens_to_ids(tokens)
                label_ids = [entity_types.index(label) for label in labels]
                # Pad input_ids and label_ids to the maximum sequence length
                padding_length = tokenizer.model_max_length - len(input_ids)
                input_ids += [tokenizer.pad_token_id] * padding_length
                label_ids += [entity_types.index('O')] * padding_length
                tokenized_data.append({'input_ids': input_ids,
                    'labels': label_ids
                })
        print(labels)
    # Convert tokenized data to PyTorch dataset
    dataset = TensorDataset(
        torch.tensor([item['input_ids'] for item in tokenized_data]), torch.tensor([item['labels'] for item in tokenized_data])
    )
    return dataset

In [7]:
import json

train_dataset = []
with open("./NER Training/doccano_export.jsonl", "r") as input:
  for line in input:
    train_dataset.append(json.loads(line))

# Prepare data for fine-tuning
train_data = tokenize_and_format_data(train_dataset, tokenizer) 
train_dataloader = DataLoader(train_data, batch_size=batch_size)

tokens ['[CLS]', 'repealed', '12', '[', 'repealed', '202', '##3', '-', '13', '-', '11', '.', ']', '[SEP]']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
tokens ['[CLS]', '1', 'subject', 'to', 'sub', '##section', '(', '3', ')', ',', 'if', 'a', 'court', 'finds', 'a', 'that', 'property', 'is', 'an', 'instrument', 'of', 'unlawful', 'activity', ',', 'and', 'b', 'that', 'a', 'person', 'is', 'an', 'un', '##in', '##vo', '##lved', 'interest', 'holder', 'with', 'respect', 'to', 'that', 'property', ',', 'the', 'court', 'must', 'make', 'the', 'orders', 'necessary', 'to', 'protect', 'the', 'interest', 'in', 'the', 'property', 'held', 'by', 'the', 'un', '##in', '##vo', '##lved', 'interest', 'holder', '.', '2', 'a', 'protection', 'order', 'issued', 'with', 'respect', 'to', 'property', 'that', 'is', 'subject', 'to', 'a', 'for', '##feit', '##ure', 'order', 'has', 'effect', 'from', 'the', 'date', 'that', 'the', 'for', '##feit', '##ure', 'order', 'is', 'effective', ',', 'unless',

In [8]:
print(train_data)

In [9]:
# Fine-tune the model
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate) 
num_epochs = 15  # Adjust as needed
for epoch in range(num_epochs):
    print('epoch')
    model.train()
    for batch in tqdm(train_dataloader, desc="Training"):
        print('batch')
        inputs, labels = batch
        # Unpack the tuple
        outputs = model(inputs, labels=labels)
        loss =  outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


epoch


Training:   0%|          | 0/8 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


batch


: 

: 

: 

In [ ]:
# Save the fine-tuned model for later 
model.save_pretrained('exported_models/fine_tuned_ner_model')
